`__file__` and `__name__` are useful dunder properties of scripts  
the following example will _NOT_ run in Jupyter because Jupyter scripts don't have `__file__` and `__name__` :'(

In [17]:
from pathlib import Path

# SELF = Path(__file__).absolute()

if __name__ == "__main__":
    print(f"Current directory is '{Path.cwd()}'")
    # print(f"Running '{SELF}'")
    print(f"'properties.py' does {'' if Path('properties.py').exists() else 'NOT '}exist in this directory.")
    # with SELF.open("r") as file:
        # print(file.read())


Current directory is '/Users/christopher/Coding/status-tool'
'properties.py' does NOT exist in this directory.


-----
## Dunder Prefix for Class Members
Python mangles dunder prefixed class members internally to prevent collisions with inheriting classes.

In [7]:
#! /usr/bin/env python3

class Parent(object):
    def __init__(self):
        self.foo = 42
        self._bar = 13
        self.__baz = 20210520
        return

    @property
    def pbaz(self):
        return self.__baz


class SeaBass(Parent):
    def __init__(self):
        super().__init__()
        self.foo = 24
        self._bar = 31
        self.__baz = 314159265
        return

    @property
    def cbaz(self):
        return self.__baz


if __name__ == "__main__":
    p = Parent()
    try:
        # _bar is squiggly because the single underscore prefix indicates an internal/private member
        print(f"Parent: {p.foo}, {p._bar}, {p.__baz}")
    except AttributeError as e:
        print(e)
        print([name for name in dir(p) if "baz" in name])
        print(f"Parent: {p.foo}, {p._bar}, {p.pbaz}")

    c = SeaBass()
    try:
        # _bar is squiggly because the single underscore prefix indicates an internal/private member
        print(f"Child: {c.foo}, {c._bar}, {c.__baz}")
    except AttributeError as e:
        print(e)
        print([name for name in dir(c) if "baz" in name])
        print(f"Child: {c.foo}, {c._bar}, {c.pbaz}, {c.cbaz}")


'Parent' object has no attribute '__baz'
['_Parent__baz', 'pbaz']
Parent: 42, 13, 20210520
'SeaBass' object has no attribute '__baz'
['_Parent__baz', '_SeaBass__baz', 'cbaz', 'pbaz']
Child: 24, 31, 20210520, 314159265


-----
## Single Underscore
Use a single underscore to capture unneeded values.

In [3]:
#! /usr/bin/env python3

for _ in range(8):
    print("Hello, World!")

vehicle = ("red", "Ford", "F150", 65536, 32768)

color, _, _, mileage, price = vehicle
print(f"The {color} vehicle has {mileage} miles on it. Sale price is ${price}.")


Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
The red vehicle has 65536 miles on it. Sale price is $32768.


-----
## Implement `__str__` for Custom String Formatting
Used by Python in `print()` and f-strings.

In [5]:
#! /usr/bin/env python3

from datetime import datetime


class Employee(object):
    def __init__(self, first, last, title, start_date):
        self._first = first
        self._last = last
        self._title = title
        self._start_date = start_date
        return

    def __str__(self):
        return f"{self._last}, {self._first}\n"\
               f"\t{self._title} ({self._start_date.strftime('%d-%b-%Y').upper()})"


if __name__ == "__main__":
    clorton = Employee("Christopher", "Lorton", "Engineering Manager", datetime(2010, 10, 11))
    print(clorton)
    print()
    print(f"{clorton}")


Lorton, Christopher
	Engineering Manager (11-OCT-2010)

Lorton, Christopher
	Engineering Manager (11-OCT-2010)


-----
## Implement `__len__` for Custom Length Calculations
Used by Python when calling `len()` on an object.

In [6]:
#! /usr/bin/env python3

from datetime import datetime
import json


class Metadata(object):
    def __init__(self, author, date, tool):
        self._author = author
        self._date = date
        self._tool = tool
        return

    def __str__(self):
        return json.dumps({"author": self._author, "date": f"{self._date.isoformat()}", "tool": self._tool})

    def __len__(self):
        return len(str(self).encode("utf-8"))


if __name__ == "__main__":
    meta = Metadata("clorton", datetime.now(), "length.py")
    # need to reserve space in the file, how much?
    length = len(meta)
    print(f"Need to reserve {length} bytes in the file for the metadata.")
    print("".join([str((i+1) % 10) for i in range(length)]))
    print(meta)


Need to reserve 80 bytes in the file for the metadata.
12345678901234567890123456789012345678901234567890123456789012345678901234567890
{"author": "clorton", "date": "2021-05-21T00:24:10.402183", "tool": "length.py"}


-----
## Implement `__call__` to Make Object Instances Callable

In [8]:
#! /usr/bin/env python3

class CountingNumbers(object):
    def __init__(self, default=0):
        self._default = default
        return

    @property
    def default(self):
        return self._default

    @default.setter
    def default(self, value):
        self._default = int(value)
        return self._default

    def __call__(self, *args, **kwargs):
        return [_ for _ in range(1, 1+(int(args[0]) if args else self._default))]


if __name__ == "__main__":
    c = CountingNumbers()
    print(f"{c()}")
    print(f"{c(13)}")
    c.default = 8
    print(f"{c()}")


[]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[1, 2, 3, 4, 5, 6, 7, 8]


-----
## Implement `__setitem__` and `__getitem__` to Create Collection Classes

In [10]:
#! /usr/bin/env python3

class CaseInsensitiveDictionary(dict):
    def __init__(self, initializer=None):
        super().__init__(initializer) if initializer is not None else super().__init__()
        return

    # ['__contains__', '__delattr__', '__delitem__', '__getattribute__', '__getitem__', '__init__', '__iter__', '__len__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__str__']
    def __setitem__(self, key, value):
        return super().__setitem__(key.lower(), value)

    def __getitem__(self, key):
        return super().__getitem__(key.lower())


if __name__ == "__main__":
    cid = CaseInsensitiveDictionary()
    cid["Christopher"] = "okay."
    cid["CHRISTOPHER"] = "yelling!"
    cid["christopher"] = "quiet."
    print(f"christopher is {cid['christopher']}")
    print(f"Christopher is {cid['Christopher']}")
    print(f"CHRISTOPHER is {cid['CHRISTOPHER']}")


christopher is quiet.
Christopher is quiet.
CHRISTOPHER is quiet.


-----
## Implement `__next__` and `__iter__` to Implement Iterable Classes

In [13]:
#! /usr/bin/env python3

"""Courtesy of ccollins@idmod.org"""


class Fib:
    def __init__(self):
        self.a, self.b = 0, 1

    def __next__(self):
        return_value = self.a
        self.a, self.b = self.b, self.a+self.b
        return return_value

    def __iter__(self):
        return self


if __name__ == "__main__":
    fibber = Fib()
    for index, value in enumerate(fibber):
        print(f"{index}: {value}")
        if index == 16:
            break


0: 0
1: 1
2: 1
3: 2
4: 3
5: 5
6: 8
7: 13
8: 21
9: 34
10: 55
11: 89
12: 144
13: 233
14: 377
15: 610
16: 987


-----
## Implement `__getstate__` and `__setstate__` for Custome Picklable Classes

In [14]:


#! /usr/bin/env python3

from io import BytesIO
import json
import pickle


class Example(object):
    def __init__(self):
        self.foo = 42
        self.bar = "All your base are belong to us!"
        return

    def __getstate__(self):
        print("In custom Example::__getstate__()...")
        state = {"oof": self.foo // 2, "rab": self.bar[::-1]}
        return state

    def __setstate__(self, state):
        print("In custom Example::__setstate__()...")
        self.foo, self.bar = state["oof"], state["rab"]
        return

    def __str__(self):
        return f"{self.__dict__}"


if __name__ == "__main__":
    JOKE = "Two birds, Pete and Repete were sitting on a fence. Pete fell off. Which one was left?"
    example = Example()
    print(f"example is {example}")

    pickled = BytesIO()
    pickle.dump(example, pickled)

    pickled.seek(0)
    test = pickle.load(pickled)
    print(f"test is a {type(test)} - {test}")


example is {'foo': 42, 'bar': 'All your base are belong to us!'}
In custom Example::__getstate__()...
In custom Example::__setstate__()...
test is a <class '__main__.Example'> - {'foo': 21, 'bar': '!su ot gnoleb era esab ruoy llA'}


-----
## `__dict__` Gives Access to Internal Object Members

In [11]:
#! /usr/bin/env python3

class Dynamic(object):
    def __init__(self):
        return

    def __setattr__(self, key, value):
        if str.islower(key[0]) and str.islower(key[-1]) and any(map(str.isupper, key)):
            self.__dict__[key] = value
        else:
            raise AttributeError
        return


if __name__ == "__main__":
    duo = Dynamic()
    print(dir(duo))
    try:
        duo.foo = "I don't think so."
    except AttributeError:
        print("Caught AttributeError setting duo.foo")
    try:
        print(f"{duo.foo}")
    except AttributeError:
        print("Caught AttributeError reading duo.foo")
    try:
        print(f"{duo.catCase=}")
    except AttributeError:
        print("Caught AttributeError reading duo.catCase")
    print("Setting duo.catCase...")
    duo.catCase = "sokoke"
    print(f"{duo.catCase=}")
    print(dir(duo))


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__']
Caught AttributeError setting duo.foo
Caught AttributeError reading duo.foo
Caught AttributeError reading duo.catCase
Setting duo.catCase...
duo.catCase='sokoke'
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'catCase']
